Streaming tables
 Materialized views
views 

In [0]:
%python

import dlt

In [0]:
select * from general_catalog.bronze_schema.order_raw

In [0]:
%python
spark.sql("USE CATALOG general_catalog")

@dlt.table(
    comment="Order Bronze table",
    table_properties={"quality": "bronze"}
)
def order_bronze():
    df = spark.readStream.table("general_catalog.bronze_schema.order_raw")
    return df
     

In [0]:
%python
@dlt.view(
    comment="Joined View"
)
def joined_vw():
    df_c = spark.read.table("Live.order_bronze")
   
   
    return df_c